# Read fire related traits from austraits data

We will download data from [AusTraits](https://austraits.org/) ([pre-print](https://www.biorxiv.org/content/10.1101/2021.01.04.425314v1)) and add entries to the database for resprouting time for each species.

Let's start loading the libraries

In [1]:
from pathlib import Path
import os
import json
import urllib
from zipfile import ZipFile
import pandas as pd
import numpy as np
import copy

## Read _austraits_ data 
We will download the file from the [Zenodo repository](https://zenodo.org/record/5112001) using the API url and saving this under the data folder.

In [2]:
repodir = Path("../../") 
dataset = "https://zenodo.org/api/records/3568417"
outputdir = repodir / "data/austraits/"

if not os.path.isdir(outputdir):
    os.makedirs(outputdir)

We use urllib to open the url and read the data (if successfully connected!)

In [3]:
def getResponse(url):
    operUrl = urllib.request.urlopen(url)
    if(operUrl.getcode()==200):
       data = operUrl.read()
    else:
       print("Error receiving data", operUrl.getcode())
    return data
zrecord = getResponse(dataset)

Response data is in json format, need to parse it and read list of files:

In [4]:
jsonData = json.loads(zrecord)
#jsonData
for files in jsonData['files']:
    print(files['key'])

austraits-3.0.2.rds
austraits-3.0.2.zip
dictionary.html
NEWS.md
readme.txt


We want to download the zip file with the csv_files

In [5]:
outputfile = outputdir / jsonData['files'][1]['key']

if os.path.isfile(outputfile):
    print('File exists')
else:
    resp = getResponse(jsonData['files'][1]['links']['self'])
    output = open(outputfile,'wb')
    output.write(resp)
    output.close()

We will read from the zipfile the data that we need:

In [6]:
zfobj = ZipFile(outputfile)
zfobj.namelist()

['austraits-3.0.2/',
 'austraits-3.0.2/taxa.csv',
 'austraits-3.0.2/methods.csv',
 'austraits-3.0.2/definitions.yml',
 'austraits-3.0.2/build_info.md',
 'austraits-3.0.2/contributors.csv',
 'austraits-3.0.2/contexts.csv',
 'austraits-3.0.2/excluded_data.csv',
 'austraits-3.0.2/traits.csv',
 'austraits-3.0.2/taxonomic_updates.csv',
 'austraits-3.0.2/sites.csv',
 'austraits-3.0.2/sources.bib']

### Read files
We will need to read the files with the definitions (in _yaml_ format), the sources or references (in _bibtex_ format) and the traits and taxonomic data (in _csv_ format)

In [7]:
import yaml

with zfobj.open('austraits-3.0.2/definitions.yml') as file:
    try:
        ATdefinitions = yaml.safe_load(file)   
        print(ATdefinitions.keys())
    except yaml.YAMLError as exc:
        print(exc)

dict_keys(['traits', 'value_type', 'austraits', 'metadata'])


In [8]:
from pybtex.database.input import bibtex
parser = bibtex.Parser()

ATrefs = parser.parse_bytes(zfobj.open('austraits-3.0.2/sources.bib').read())


In [9]:
ATtraits = pd.read_csv(zfobj.open('austraits-3.0.2/traits.csv'),low_memory=False)

In [12]:
ATtaxa = pd.read_csv(zfobj.open('austraits-3.0.2/taxa.csv'))

We will also read the updated species data from BioNET:

In [13]:
inputdir = repodir / "data/"
BioNET = pd.read_excel(inputdir / 'vis-survey-datasheet-6000.PowerQuery.20210708.xlsx')

In [26]:
ATtraits.trait_name.unique().size
ATtraits.taxon_name.unique().size
#ATtraits.shape

28640

In [25]:
ATtraits.head()

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
0,Ahrens_2019,Corymbia calophylla,BOO,NaN,Ahrens_2019_001,leaf_area,12883.7,mm2,2017-03,raw_value,1,Corymbia calophylla
1,Ahrens_2019,Corymbia calophylla,BOO,NaN,Ahrens_2019_001,leaf_delta13C,-27.98,per mille,2017-03,raw_value,1,Corymbia calophylla
2,Ahrens_2019,Corymbia calophylla,BOO,NaN,Ahrens_2019_001,leaf_N_per_dry_mass,9,mg/g,2017-03,raw_value,1,Corymbia calophylla
3,Ahrens_2019,Corymbia calophylla,BOO,NaN,Ahrens_2019_001,leaf_PRI,0.022642696,umol/umol,2017-03,raw_value,1,Corymbia calophylla
4,Ahrens_2019,Corymbia calophylla,BOO,NaN,Ahrens_2019_001,modified_NDVI,0.711535497,dimensionless,2017-03,raw_value,1,Corymbia calophylla


## Utility functions

In [12]:
def extract_reflabel(refid):
    authors=list()
    year=ATrefs.entries[refid].fields['year']
    for person in ATrefs.entries[refid].persons['author']:
        authors.extend(person.last_names)
    reflabel = "%s %s" % (" ".join(authors),year)
    if len(reflabel)>50:
        reflabel=reflabel[0:47]+"..."
    return(reflabel)

def extract_refinfo(refid):
    year=ATrefs.entries[refid].fields['year']
    title=ATrefs.entries[refid].fields['title']
    persons = ATrefs.entries[refid].persons['author']
    if len(persons)==1:
        refcitation = "%s (%s) %s" % (persons[0],year, title)
    else:
        authors=list()
        for person in persons:
            authors.append(person.__str__())
        refcitation = "%s (%s) %s" % ("; ".join(authors),year, title)
    for f in ('journal','volume','doi'):
        if f in ATrefs.entries[refid].fields.keys():
            refcitation = refcitation + " " + ATrefs.entries[refid].fields[f]
    return refcitation 

def match_spcode(row):
    spname=row['taxon_name']
    altname=row['original_name']
    result={'species':spname}
    if altname!=spname:
        result['original_notes']=['original_name:',altname]
    spp_info = BioNET[BioNET['scientificName'] == spname] 
    spcode=None
    if len(spp_info)==1 and spp_info.speciesCode_Synonym is not None:
        spcode=spp_info.speciesCode_Synonym.values[0]
        result['species_code']=spcode
    elif spname != altname:
        spp_info = BioNET[BioNET['scientificName'] == altname]
        if len(spp_info)==1 and spp_info.speciesCode_Synonym is not None:
            spcode=spp_info.speciesCode_Synonym.values[0]
            result['species_code']=spcode
            result['original_notes'].append('original name used to match with BioNET names')
 
    return result


In [13]:
print(extract_refinfo('NSWFRD_2014'))
print(extract_reflabel('NSWFRD_2014'))

Kenny, Belinda; Orscheg, Corinna; Tasker, Elizabeth; Gill, Malcolm A.; Bradstock, Ross (2014) {NSW Flora Fire Response Database, v2.1}
Kenny Orscheg Tasker Gill Bradstock 2014


In [14]:
def create_record(row):
    refid=row['dataset_id']
    val = row['value']
    records=list()
    w = 0
    wnote = "incomplete record"
    notes = list()
    record={'main_source': 'austraits-3.0.2',
            'additional_notes': ['Reclassification rules provided by DAK',
                                'Automatic extraction with python script'],
            'raw_value': [row['trait_name'],val,row['value_type']],
            'original_notes': list()}
    if row['site_name'] != "nan":
        notes.append('site name:')
        notes.append(row['site_name'])
    if refid in (ATrefs.entries.keys()):
        reflabel = extract_reflabel(refid)
        record['original_sources']=[reflabel]
        if reflabel in ('NSWFRD_2014','Kenny Orscheg Tasker Gill Bradstock 2014'):
            w=0
            wnote="default of 0 for redundant records"
        else:
            w=1
            wnote="default of 1"
    spinfo=match_spcode(row)
    for key in spinfo.keys():
        record[key]=spinfo[key]
    record['weight']=w
    record['weight_notes'] = ["python-script import",wnote]
    
    if len(notes)>0:
        record['original_notes']=notes
        
    for sw in val.split(" "):
        indrecord=copy.deepcopy(record)
        sw=sw.strip(" ")
        transvalue=switcher.get(sw, None)
        if sw != val:
            indrecord['raw_value'].extend(['->',sw])            
        if transvalue is not None:   
            indrecord["norm_value"]=transvalue
        records.append(indrecord)
    return(records)

## Categorical traits

In [15]:
alltraits = ATtraits['trait_name'].unique()
len(alltraits)

448

In [16]:
#import re
for item in alltraits:
    y = item.find("disp")
    if y>-1:
        print(item)


dispersal_appendage
dispersal_syndrome
dispersers


### Dormancy type

In [17]:
element=ATdefinitions['traits']['elements']['dormancy_type']
for k in element.keys():
    print("{key} :: {value}".format(key=k,value=element[k]))
#    print('' % (ATdefinitions['traits']['elements']['dormancy_type']['values'])

description :: Classification for seed dormancy
type :: categorical
label :: Dormancy type
values :: {'morphophysiological_dormancy': 'Seeds exhibit morphophysiological dormancy', 'non_dormant': 'Seeds are non-dormant', 'physical_dormancy': 'Seeds exhibit physical dormancy', 'physiological_dormancy': 'Seeds exhibit physiological dormancy'}


In [18]:
ss = (ATtraits['trait_name']=='dormancy_type' )
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
479894,Ooi_2007,Acacia binervata,Fredericktown,NaN,Ooi_2007_00001,dormancy_type,physical_dormancy,NaN,1978-11-21,expert_mean,NaN,Acacia binervata
481466,Ooi_2007,Angophora bakeri,Agnes Banks to Castlereagh,NaN,Ooi_2007_01580,dormancy_type,non_dormant,NaN,1977-02-18,expert_mean,NaN,Angophora bakeri
488008,Ooi_2007,Isopogon anemonifolius,Cordeaux Cataract Catchment,NaN,Ooi_2007_08122,dormancy_type,physiological_dormancy,NaN,1975-08-12,expert_mean,NaN,Isopogon anemonifolius
489445,Ooi_2007,Bulbine bulbosa,Blacktown,NaN,Ooi_2007_09558,dormancy_type,morphophysiological_dormancy,NaN,1974-12-04,expert_mean,NaN,Bulbine bulbosa
489745,Ooi_2007,Calotis cuneifolia,Ashford,NaN,Ooi_2007_09858,dormancy_type,non_dormant physiological_dormancy,NaN,1978-11-25,expert_mean,NaN,Calotis cuneifolia


This data is incomplete, the full dataset is in a separate branch that will be released later on. 
In the mean time we will use this version provided by Elizabeth Wenk:

In [19]:
dormancy = pd.read_csv(repodir/'data/austraits/dormancy_type data from Austraits.csv')

In [20]:
dormancy

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
0,Ooi_2007,Acacia binervata,Fredericktown,NaN,Ooi_2007_00001,dormancy_type,physical_dormancy,NaN,1978-11-21,expert_mean,NaN,Acacia binervata
1,Ooi_2007,Acacia caesiella,Munghorn Gap,NaN,Ooi_2007_00101,dormancy_type,physical_dormancy,NaN,1979-02-07,expert_mean,NaN,Acacia caesiella
2,Ooi_2007,Acacia elata,Nettle Hill,NaN,Ooi_2007_00201,dormancy_type,physical_dormancy,NaN,1978-12-09,expert_mean,NaN,Acacia elata
3,Ooi_2007,Acacia elongata,Richmond - Penrith,NaN,Ooi_2007_00301,dormancy_type,physical_dormancy,NaN,1974-12-04,expert_mean,NaN,Acacia elongata
4,Ooi_2007,Acacia falcata,Kurri Kurri,NaN,Ooi_2007_00401,dormancy_type,physical_dormancy,NaN,1978-12-02,expert_mean,NaN,Acacia falcata
...,...,...,...,...,...,...,...,...,...,...,...,...
173,Ooi_2007,Stylidium graminifolium,Myall Lakes,NaN,Ooi_2007_14300,dormancy_type,morphophysiological_dormancy,NaN,1976-03-08,expert_mean,NaN,Stylidium graminifolium
174,Ooi_2007,Trachymene incisa subsp. incisa,Myall Lakes,NaN,Ooi_2007_14400,dormancy_type,morphophysiological_dormancy,NaN,1975-05-23,expert_mean,NaN,Trachymene incisa var incisa
175,Ooi_2007,Westringia fruticosa,Newport Beach,NaN,Ooi_2007_14527,dormancy_type,physiological_dormancy,NaN,1974-04-02,expert_mean,NaN,Westringia fruticosa
176,Ooi_2007,Woollsia pungens,Ku-ring-gai Chase National Park,NaN,Ooi_2007_14548,dormancy_type,physiological_dormancy,NaN,1974-11-08,expert_mean,NaN,Woollsia pungens


In [22]:
dormancy.fillna("nan",inplace=True)
#target = ATtraits[ss]
target=dormancy
switcher={
        "non_dormant": "ND",
        'physiological_dormancy': "PD",
        'morphophysiological_dormancy': 'MPD', 
        'physical_dormancy': 'PY'
    }
    
reflist=list()
records=list()
for idx, row in target.iterrows():
    record=create_record(row)
    refid=row['dataset_id']
    extract_reflabel(refid)
    if refid not in reflist:
        reflist.append(refid)
    records.extend(record)

len(records)

181

In [23]:
print(records[10])

{'main_source': 'austraits-3.0.2', 'additional_notes': ['Reclassification rules provided by DAK', 'Automatic extraction with python script'], 'raw_value': ['dormancy_type', 'physical_dormancy', 'expert_mean'], 'original_notes': ['site name:', 'Kurnell'], 'original_sources': ['Ooi Myerscough Auld 2007'], 'species': 'Acacia longifolia', 'species_code': '3816', 'weight': 1, 'weight_notes': ['python-script import', 'default of 1'], 'norm_value': 'PY'}


In [24]:
from configparser import ConfigParser
import psycopg2
from psycopg2.extensions import AsIs

filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'

parser = ConfigParser()
parser.read(filename)

dbparams = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        dbparams[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

In [25]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**dbparams)
cur = conn.cursor()
affected_rows=0

        
for refid in reflist:
    cur.execute("INSERT INTO litrev.ref_list(ref_code,alt_code,ref_cite) values(%s,%s,%s) ON CONFLICT DO NOTHING",
                (extract_reflabel(refid), refid, extract_refinfo(refid)))
    affected_rows = affected_rows+cur.rowcount
conn.commit()
print("total number of lines updated: %s" % affected_rows)

insert_statement = 'insert into litrev.germ8 (%s) values %s ON CONFLICT DO NOTHING'
print("total of %s records prepared" % len(records)) 
for record in records: 
    cur.execute(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values())))
    affected_rows = affected_rows+cur.rowcount
records.clear()
conn.commit()
print("total number of lines updated: %s" % affected_rows)

cur.close()
if conn is not None:
    conn.close()
    print('Database connection closed.')     


Connecting to the PostgreSQL database...
total number of lines updated: 0
total of 181 records prepared
total number of lines updated: 181
Database connection closed.


### Other traits


#### Extracted by Will
These were suggested by Will in April 2021


In [ ]:
traits=('seed_mass','germination_treatment','dispersal_syndrome','vegetative_regeneration',
        'fire_and_establishing','fire_response_on_maturity','lifespan',
        'seed_storage_location',
        'regen_strategy',
        'bark_thickness','serotiny','soil_seedbank','time_from_fire_to_fruit',
        'fire_response_juvenile','fire_response','fire_cued_seeding','resprouting_proportion_individuals'


#### Dispersal

In [146]:
traits=('dispersal_appendage','dispersers', 'dispersal_syndrome')
for trait in traits:
    ss =(ATtraits['trait_name']==trait)
    #print(trait)
    #print(ATtraits[ss].value.unique())

ATtraits[ss].groupby('value')['taxon_name'].count()

value
adhesion                                130
adhesion wind                             2
anemochory                             2739
anemochory animal_vector                  1
anemochory animal_vector hydrochory       1
                                       ... 
vertebrate                              129
vertebrate water wind                     1
water                                    91
water wind                               15
wind                                   1158
Name: taxon_name, Length: 72, dtype: int64

In [147]:
element=ATdefinitions['traits']['elements']['dispersal_syndrome']['values']
for k in element.keys():
    print("{key} ".format(key=k,value=element[k]))
   #print("{key} :: {value}".format(key=k,value=element[k]))


adhesion 
anemochory 
animal_vector 
aril 
atelochory 
ballistic 
barochory 
barochory_small_seeds 
bird 
chamaechory 
dispersal_rare 
dyszoochory 
elaiosome 
endozoochory 
endozoochory_mammal 
endozoochory_bird 
exozoochory 
exozoochory_mammal 
exozoochory_bird 
fleshy_fruit 
fleshy_wings_capsule 
gravity 
hydrochory 
insect 
invertebrate 
invertebrate_insect 
mammal 
mobile 
mud_on_cars 
myrmecochory 
nautohydrochory 
ombrohydrochory 
parent_plant_or_diaspore 
synzoochory 
unassisted 
undefined 
vertebrate 
water 
wind 
xerochasywater 


In [156]:
ss =ATtraits['trait_name']=='dispersal_appendage'
target = ATtraits[ss]
switcher={
        "aril": "ant",
"awns": "animal-cohesion",
"awn_bristle": "animal-cohesion",
"barbs": "animal-cohesion",
"beak": "animal-cohesion",
"berry": "animal-ingestion",
"caruncle": "animal-cohesion",
"curved_awn": "animal-cohesion",
"drupe": "animal-ingestion",
"elaiosome": "ant",
"glumes": "wind-hairs",
"plumose": "wind-hairs",
"pseudo-wing": "wind-wing",
"receptacle": "wind-wing",
"seed_airsac": "wind-wing",
"seed_unilaterally_winged": "wind-wing",
"seed_wing_obsolete": "wind-wing",
"winged_fruit": "wind-wing",
"wings": "wind-wing",
"wings_small": "wind-wing",
"floating seed": "water"}

    
reflist=list()
records=list()
for idx, row in target.iterrows():
    record=create_record(row)
    refid=row['dataset_id']
    extract_reflabel(refid)
    if refid not in reflist:
        reflist.append(refid)
    records.extend(record)

In [157]:
ss =ATtraits['trait_name']=='dispersers'
target = ATtraits[ss]
switcher={
"ants": "ant", 
"bats": "animal-unspec.", 
"birds": "animal-unspec.", 
"cassowary": "animal-unspec.", 
"flying": "animal-unspec.", 
"flying_foxes": "animal-unspec.", 
"mammals": "animal-unspec.", 
"non-flying": "animal-unspec.", 
"rodents": "animal-unspec.", 
"vertebrate": "animal-unspec.", 
"wind": "wind-unspec.", 
"water": "water", 
}
    
for idx, row in target.iterrows():
    record=create_record(row)
    refid=row['dataset_id']
    extract_reflabel(refid)
    if refid not in reflist:
        reflist.append(refid)
    records.extend(record)

In [158]:
ss =ATtraits['trait_name']=='dispersal_syndrome'
target = ATtraits[ss]
switcher={
"adhesion": "animal-cohesion",
"anemochory": "wind-unspec.",
"animal_vector": "animal-unspec.", 
"aril": "ant",
"ballistic": "ballistic", 
"bird": "animal-unspec.", 
"dispersal_rare": "passive",
"dyszoochory": "animal-ingestion",
"elaiosome": "ant",
"endozoochory": "animal-ingestion",
"endozoochory_mammal": "animal-ingestion",
"endozoochory_bird": "animal-ingestion",
"exozoochory": "animal-cohesion",
"exozoochory_mammal": "animal-cohesion",
"exozoochory_bird": "animal-cohesion",
"gravity":"passive",
"hydrochory":"water",
"insect": "ant",
"invertebrate_insect": "ant", 
"mammal": "animal-unspec.", 
"myrmecochory": "ant", 
"nautohydrochory": "water", 
"ombrohydrochory": "water",
"synzoochory": "animal-unspec.",  
"unassisted": "passive",
"vertebrate": "animal-unspec.", 
"water": "water",
"wind": "wind-unspec.", 
}
    
for idx, row in target.iterrows():
    record=create_record(row)
    if 'dataset_id' in row.keys():
        refid=row['dataset_id']
        #extract_reflabel(refid)
        if refid not in reflist:
            reflist.append(refid)
    records.extend(record)

In [159]:
'Williams_2011' in (ATrefs.entries.keys())

False

In [160]:
print(len(records))
#records[1265]
len(reflist)

19321


38

In [164]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**dbparams)
cur = conn.cursor()
affected_rows=0

        
for refid in reflist:
    if refid in (ATrefs.entries.keys()):
        cur.execute("INSERT INTO litrev.ref_list(ref_code,alt_code,ref_cite) values(%s,%s,%s) ON CONFLICT DO NOTHING",
                (extract_reflabel(refid), refid, extract_refinfo(refid)))
        affected_rows = affected_rows+cur.rowcount

conn.commit()
print("total number of lines updated: %s" % affected_rows)

insert_statement = 'insert into litrev.disp1 (%s) values %s ON CONFLICT DO NOTHING'
print("total of %s records prepared" % len(records)) 
for record in records: 
    cur.execute(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values())))
    affected_rows = affected_rows+cur.rowcount
records.clear()
conn.commit()
print("total number of lines updated: %s" % affected_rows)

cur.close()
if conn is not None:
    conn.close()
    print('Database connection closed.')     


Connecting to the PostgreSQL database...
total number of lines updated: 21
total of 19321 records prepared
total number of lines updated: 19342
Database connection closed.


In [162]:
refid


'Williams_2011'

### References

In [117]:
#bib_data.entries.keys()
for key in bib_data.entries.keys():
    print(key)
bib_data.entries['Adams_1984']
# dir(bib_data.entries['Adams_1984'])

Adams_1984
Ahrens_2019
ANBG_2019
Angevin_2011
Apgaua_2015
Apgaua_2017
Ashton_1975
Ashton_1976
Atkinson_2020
Atkinson_2020_2
Attiwill_1980
Baker_2019
Barlow_1981
Bean_1997
Bell_1985
Bennett_1997
Bevege_1978
Birk_1992
Blackman_2010
Blackman_2014
Blackman_2018
Bloomfield_2018
Bolza_1975
Bragg_2002
BRAIN_2007
Briggs_2010
Brock_1993
Brodribb_2009
Buckton_2019
Burgess_2007
Burrows_2001
Butler_2011
CAB_2009
Caldwell_2016
Canham_2009
Carpenter_1994
Carpenter_2005
Catford_2011
Catford_2014
Cernusak_2006
Cernusak_2011
Chandler_2002
Chave_2009
Cheal_2017
Cheesman_2020
Chen_2017
Chinnock_2007
Choat_2005
Choat_2006
Choat_2012
Chudnoff_1984
CIRAD_2009
Clarke_2015
Cooper_2004
Cooper_2013
Cornwell_2006
CPBR_2002
Craven_1987
Craven_2010
Crisp_2017
Cromer_1975
Cross_2009
Crous_2013
Crous_2019
Cunningham_1999
Curran_2009
Curtis_2012
Denton_2007
Desh_1996
Detombeur_2021
Dong_2017
Dong_2020
Du_2018
Du_2019
Duan_2015
Duncan_1998
Duncan_2011
Dwyer_2017
Dwyer_2018
Eamus_1998
Eamus_1999
Eamus_1999_2
Edwards_20

Entry('article',
  fields=[
    ('year', '1984'), 
    ('journal', 'Australian Journal of Botany'), 
    ('title', '{Role of Acacia spp. in nutrient balance and cycling in regenerating Eucalyptus regnans F. Muell. forests. I. Temporal changes in biomass and nutrient content}'), 
    ('volume', '32'), 
    ('number', '2'), 
    ('pages', '205--215'), 
    ('doi', '10.1071/bt9840205')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Adams, M. A.'), Person('M, P.'), Person('{Attiwill}')])]))

In [176]:
print(bib_data.entries['Adams_1984'].fields['year'])
##''.join(person for person in bib_data.entries['Adams_1984'].persons['author'])
for person in bib_data.entries['Adams_1984'].persons['author']:
    print(person.__str__())

dir(person)


1984
Adams, M. A.
M, P.
{Attiwill}


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parse_string',
 'bibtex_first',
 'bibtex_first_names',
 'first',
 'first_names',
 'get_part',
 'get_part_as_text',
 'last',
 'last_names',
 'lineage',
 'lineage_names',
 'middle',
 'middle_names',
 'prelast',
 'prelast_names',
 'rich_first_names',
 'rich_last_names',
 'rich_lineage_names',
 'rich_middle_names',
 'rich_prelast_names',
 'style1_re',
 'style2_re',
 'valid_roles']